In [1]:
import psycopg2
from psycopg2 import OperationalError

import pandas as pd
import numpy as np
import matplotlib.pylab as plt

import locale # Converter string com € em float

from IPython.display import HTML

***
    
    Ligação à base de dados

***

In [2]:
conn = psycopg2.connect(
    host = "contratos-base-gov1.cf87yxnqgph8.eu-central-1.rds.amazonaws.com",
    port = 5432,
    #database = "contratosbasegov",
    user = "contratosbasegov",
    password = "8n9nyeTBFUyCcLJShNrZdPUai2KQkue4")

cur = conn.cursor()

***

    Executar este comando quando ocorrer um erro ao tentar realizar uma query

***

In [3]:
cur.execute('''ROLLBACK;''')

***

# Funções


***

    ID's dos contratos

In [4]:
def all_ids(table):
    '''
    Função que retorna todos os ids dos contratos de uma tabela

    Parâmetros :    
        - table : tabela de interesse

    Return : 
        - list : ids de todos os contratos de uma tabela 
    '''

    cur = conn.cursor()
    cur.execute(''' 
                SELECT id
                FROM "{}"; '''.format(table)) 
    return list(cur.fetchall())


ids = all_ids("contratos")

    Nome das colunas

In [5]:
def col_names(table):
    '''
    Função que retorna os nomes das colunas de uma tabela
    
    Parâmetros : 
        - table : tabela de interesse

    return : 
        - pandas DataFrame : nomes das colunas
    '''

    cur = conn.cursor()
    cur.execute('''
                SELECT column_name
                FROM information_schema.columns
                WHERE table_name = %s;''', (table,))

    cnames = pd.DataFrame(cur.fetchall())
    #cnames = [row[0] for row in cur.fetchall()]
    return cnames

HTML(pd.DataFrame(col_names('contratos')).to_html(index=True))

,0
0,id
1,n_anuncio
2,anuncio_preco_base
3,anuncio_proposalDeadline
4,anuncio_cnccs
5,anuncio_contractingProcedureAliasID
6,anuncio_contractingProcedureId
7,anuncio_drPublicationDate
8,anuncio_dreNumber
9,anuncio_dreSeries


    Número de Contratos


In [6]:
def n_contracts(table):
    '''
    Retorna o número de contratos de uma tabela pertencente à base de dados

    Parâmetros : 
        - table : tabela de interesse

    return : 
        - int : número de contrato
    '''
    
    cur = conn.cursor()
    cur.execute('''
                SELECT COUNT(*) 
                FROM "{}"; '''.format(table))
    ncontract = cur.fetchone()[0]
    return ncontract

n_contracts("contratos")

2184

    Função que retorna contrato para número de id

In [7]:
def contrato(ide):
    '''
    '''
        
    cur = conn.cursor()
    cur.execute('''
        SELECT *
        FROM "contratos"
        WHERE id = %s; ''', (ide,))
    return pd.DataFrame(cur.fetchall())


def contratos(ide):
    '''
    '''
        
    cur = conn.cursor()
    cur.execute('''
        SELECT *
        FROM "contratos"
        WHERE id IN %s; ''', (tuple(ide),))
    return pd.DataFrame(cur.fetchall())

    Preço contratual

In [8]:
def preco_contrato1(ide):
    '''
    Função que retorna preço contratual a partir do id do anúncio para a tabela "contratos"

    Parâmetros :
        - ide : id do anúncio

    Return : 
        - int : preço contratual
    '''
        
    cur = conn.cursor()
    cur.execute('''
        SELECT preco_contratual
        FROM "contratos"
        WHERE id = %s; ''', (ide,))
    return float(cur.fetchone()[0])



# ---------------------------------------------------------------------------------------------------------------------------------------------------------



def preco_contrato2(ide, table = ""):
    '''
    Função que retorna preço contratual a partir do id do anúncio para um dada tabela

    Parâmetros :
        - ide : id do anúncio
        - table : tabela de interesse. Caso input esteja vazio, usar tabela 'contratos'

    Return : 
        - int : preço contratual
    '''

    if table == "": 
        table = "contratos"
        
    cur = conn.cursor()
    cur.execute('''
        SELECT preco_contratual
        FROM "{}"
        WHERE id = %s; '''.format(table), (ide,))
    return float((cur.fetchone())[0])


    
# ---------------------------------------------------------------------------------------------------------------------------------------------------------
    


#def preco_contrato3(id_anuncio):
#    
#    '''
#    Função que retorna preço contratual a partir de uma lista de ids de anúncios
#    '''
#    
#    cur.execute('''
#        SELECT preco_contratual
#        FROM "contratos"
#        WHERE id IN %s; ''', (tuple(id_anuncio),))
#    
#    return np.array(cur.fetchall()[0])

def preco_contrato3(id_anuncio):
    
    '''
    Função que retorna preço contratual a partir de uma lista de ids de anúncios
    '''
    
    cur.execute('''
        SELECT preco_contratual
        FROM "contratos"
        WHERE id IN %s; ''', (tuple(id_anuncio),))

    preco = list(cur.fetchall())
    
    n = len(preco)
    p = np.zeros(n)

    for i in range(n):
        p[i] = (preco[i][0]).replace(".", "").replace(",",".").replace("€","")

    return p


    
# ---------------------------------------------------------------------------------------------------------------------------------------------------------
    



def preco_contrato4(ide, table = ""):
    '''
    Função que retorna preço contratual a partir de uma lista de ids de anúncios para uma determinada tabela

    Parâmetros :
        - ide : id do anúncio
        - table : tabela de interesse. Caso input esteja vazio, usar tabela 'contratos'

    Return : 
        - int : preço contratual
    '''

    
    if table == "": 
        table = "contratos"

    
    cur.execute('''
        SELECT preco_contratual
        FROM "{}"
        WHERE id IN %s; '''.format(table), (tuple(ide),))
    
    return np.asarray(cur.fetchall())
    

    
# ---------------------------------------------------------------------------------------------------------------------------------------------------------

    Preço Base

In [9]:
def preco_base1(ide):
    '''
    Função que retorna preço base a partir do id do anúncio para a tabela "contratos"

    Parâmetros :
        - ide : id do anúncio

    Return : 
        - int : preço contratual
    '''
        
    cur = conn.cursor()
    cur.execute('''
        SELECT anuncio_preco_base
        FROM "contratos"
        WHERE id = %s; ''', (ide,))


    preco = cur.fetchone()[0]
    p1 = float(preco[:-2].replace(".", "").replace(",","."))
    
    return p1



# ---------------------------------------------------------------------------------------------------------------------------------------------------------



def preco_base2(ide, table = ""):
    '''
    Função que retorna preço base a partir do id do anúncio para um dada tabela

    Parâmetros :
        - ide : id do anúncio
        - table : tabela de interesse. Caso input esteja vazio, usar tabela 'contratos'

    Return : 
        - int : preço contratual
    '''

    if table == "": 
        table = "contratos"


    cur = conn.cursor()
    cur.execute('''
        SELECT anuncio_preco_base
        FROM "{}"
        WHERE id = %s; '''.format(table), (ide,))


    preco = cur.fetchone()[0]
    p1 = float(preco[:-2].replace(".", "").replace(",","."))
    
    return p1
    


# ---------------------------------------------------------------------------------------------------------------------------------------------------------
    


def preco_base3(id_anuncio):
    
    '''
    Função que retorna preço base a partir de uma lista de ids de anúncios
    '''
    
    cur.execute('''
        SELECT anuncio_preco_base
        FROM "contratos"
        WHERE id IN %s; ''', (tuple(id_anuncio),))

    preco = list(cur.fetchall())


    # Como os valores do preco base estão no formato ---.---,--€ é precio converter em -------.-- para comparar posteriormente
    n = len(preco)
    p = np.zeros(n)
    
    for i in range(n):

        if preco[i][0] != 'None':
            p[i] = (preco[i][0]).replace(".", "").replace(",",".").replace("€","")

        else:
            pass
            
    return p


a = preco_base3(ids)

#pd.DataFrame(a).describe().apply(lambda s: s.apply(lambda x: format(x, 'g')))
print("Nr de Zero Values :", len(a) - (np.count_nonzero(a)))
print("Nr de Non Zero Values :", (np.count_nonzero(a)))
print("% non zero values : ",round(np.count_nonzero(a)/len(a) * 100,1))

Nr de Zero Values : 1840
Nr de Non Zero Values : 344
% non zero values :  15.8


In [10]:
nn_index = np.where(a != 0)[0]                 # índices dos ids dos contratos com preço base não nulo

id1 = np.array(ids)                            # Transformação da lista de ids em array. Assim podemos extrair os ids não nulos usando os índices acima

ids_nn = tuple(map(tuple, id1[nn_index]))      # ids não nulos. É preciso converter em tuplo para usar como input na função de preço base

pb = preco_base3(ids_nn)

pc = preco_contrato3(ids_nn)

In [39]:
def redflag(pbase, pcontr, tol, ids):
    
    """
    Função que calcula o preço base e preço contratual de um contrato realizadp
    Se o preço contratual estiver contido num intervalo em torno do preço base é levantada um flag
    O intervalo é definido pelo parâmetro tolerância e é definido como : [preço_base - preço_base*tolerância, preço_base + preço_base*tolerância]

    Parâmetros de entrada : 
        pbase : array com os preços base
        pcontr : array com os preços contratuas
        tol : valor da tolerância. Só pode tomar valores entre 0 e 1
        ids : id's dos contratos em questão

    Return : 
        f : tuplo com os id's dos contratos com flag associada
    """

    # Garantir que dimensão dos arrays com os preços é igual
    if len(pbase) != len(pcontr):
        return "Error : dim pbase != dim pcontr"

    # Garantir que tolerância é um numero entre 0 e 1
    if tol < 0 or tol > 1 :
        return "Error : tolerance must belong between 0 and 1"

    # Número de preços base
    n = len(pbase)

    # Array que guarda ocorrência - ou não - de uma flag
    flags = np.zeros(n)

    for i in range(n):

        # Definir limites superior e inferior, respetivamente
        up_lim = pbase[i] + tol*pbase[i]
        lo_lim = pbase[i] - tol*pbase[i]

        if pcontr[i] <= up_lim and pcontr[i] >= lo_lim:
            flags[i] = 1
            
        else:
            pass


    # Conversão do tuplo de ids num array de uma coluna
    ids = np.array(ids).reshape((n,))

    # Contratos com ocorrência de uma flag
    pos = np.where(flags != 0)

    # Selecionar contratos onde ocorre flag
    f = ids[pos]
    f = f.reshape((len(f),1))

    # Conversão do conjunto de contratos em tuplo para poder usar como input nas funções que têm como input id's de contratos
    f = tuple(map(tuple,f))
    
    return f


coiso = redflag(pb,pc,.1, ids_nn)
print(coiso)

contratos(coiso).head(5)

(('10328343',), ('10328309',), ('10328297',), ('10328300',), ('10327997',), ('10327910',), ('10327868',), ('10327846',), ('10327805',), ('10327793',), ('10327667',), ('10327643',), ('10326939',), ('10326608',), ('10327853',))


,0,1,2,3,4,5,6,7,8,9,...,51,52,53,54,55,56,57,58,59,60
0,10326608,13580/2023,226000.00,30.,False,6646356,13325647,2023-08-10,155,2,...,None,None,False,None,True,None,Código dos Contratos Públicos (DL111-B/2017) e...,None,None,1699372102
1,10326939,2041/2023,74018.00,3.,False,6669486,13370711,2023-09-08,175,2,...,None,None,False,None,True,None,Código dos Contratos Públicos (DL111-B/2017) e...,None,None,1699371520
2,10327643,15041/2023,249985.47,30,False,6688208,13409402,2023-09-11,176,2,...,None,None,False,None,True,None,Código dos Contratos Públicos ( DL 111-B/2017 ),None,None,1699377558
3,10327667,13124/2023,640000.00,30,False,6638403,13305568,2023-08-03,150,2,...,None,None,False,None,True,None,Código dos Contratos Públicos (DL 111-B/2017) ...,None,None,1699377516
4,10327793,13939/2023,443636.64,30,False,6652359,13339665,2023-08-18,160,2,...,None,None,False,None,True,None,Código dos Contratos Públicos (DL111-B/2017) e...,None,None,1699377348


<br>

<br>

<br>

<br>

<br>

In [83]:
cur = conn.cursor()
cur.execute('''
        SELECT tipo_procedimento, COUNT (tipo_procedimento)
        FROM "contratos"
        GROUP BY "tipo_procedimento"
        ORDER BY COUNT(tipo_procedimento) DESC;;''')

df = (pd.DataFrame(cur.fetchall()))
HTML(df.to_html(index=False))

0,1
Ajuste Direto Regime Geral,773
Consulta Prévia,253
Ao abrigo de acordo-quadro (art.º 259.º),234
Concurso público,203
Ao abrigo de acordo-quadro (art.º 258.º),36
Consulta Prévia Simplificada,6
Concurso limitado por prévia qualificação,4
Procedimento de negociação,2


In [92]:
cur = conn.cursor()

cur.execute('''
        SELECT "contractTypes", COUNT ("contractTypes")
        FROM "contratos"
        GROUP BY "contractTypes"
        ORDER BY  COUNT("contractTypes") DESC;;''')

df = (pd.DataFrame(cur.fetchall()))
HTML(df.to_html(index=False))

0,1
Aquisição de bens móveis,836
Aquisição de serviços,535
Empreitadas de obras públicas,118
Locação de bens móveis,8
Aquisição de bens móveis<br/>Aquisição de serviços,6
Aquisição de serviços<br/>Aquisição de bens móveis,3
Concessão de obras públicas,2
Concessão de serviços públicos,1
Aquisição de serviços<br/>Empreitadas de obras públicas<br/>Aquisição de bens móveis,1
Aquisição de serviços<br/>Empreitadas de obras públicas,1


In [7]:
cur.execute('''
    TABLE "contratos";
''')

df = (pd.DataFrame(cur.fetchmany(1)))
HTML(df.to_html(index=False))

0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61
10323526,9385/2023,"1.893.000,00 €",25 dias.,False,6549928,13118913,06-06-2023,109,2,False,False,https://dre.pt/application/file/214106482,Anúncio de procedimento,Concurso público,Requalificação e Ampliação das Piscinas Municipais de Fânzeres,02-11-2023,16-10-2023,1863547.21,366,Município de Gondomar (506848957)(https://www.base.gov.pt/Base4/pt/detalhe/?type=entidades&id=173),"Artigo 19.º, alínea b) do Código dos Contratos Públicos","Ivo Ribeiro, Unipessoal, Lda. (Electroinstal) (507655745)(https://www.base.gov.pt/Base4/pt/detalhe/?type=entidades&id=32793)","Arpecdouro S.A (508263425)|||Edilages, S.A. (508559871)|||Décio Soares , Unipessoal, Lda. (509274374)",https://www.base.gov.pt/Base4/pt/detalhe/?type=anuncios&id=348901,https://community.vortal.biz/PRODPublic/Tendering/OpportunityDetail/Index?noticeUID=PT1.NTC.2420642,https://www.base.gov.pt/Base4/pt/resultados/?type=doc_documentos&id=2071195&ext=.pdf,false,45453000-7,False,None,None,False,False,None,False,None,False,Empreitadas de obras públicas,Obras de revisão e recuperação,Principal,1863547.21 €,Requalificação e Ampliação das Piscinas Municipais de Fânzeres,Não aplicável,None,366 dias,"Portugal, Porto, Gondomar",Não aplicável.,Não aplicável.,None,False,False,None,None,False,,True,None,"Código dos Contratos Públicos (DL111-B/2017) e Lei n.º 30/2021, de 21.05",None,None,1699032847


In [35]:
cur.execute('''
    SELECT *
    FROM "contratos" LIMIT 30;
''')

df = pd.DataFrame(cur.fetchall())
df

,0,1,2,3,4,5,6,7,8,9,...,52,53,54,55,56,57,58,59,60,61
0,10323526,9385/2023,"1.893.000,00 €",25 dias.,False,6549928,13118913,06-06-2023,109,2,...,None,None,False,,True,None,Código dos Contratos Públicos (DL111-B/2017) e...,None,None,1699032847
1,10325039,4400/2023,"143.100,00 €",9 dias.,True,6405891,12822340,21-03-2023,57,2,...,None,None,False,,True,None,Código dos Contratos Públicos (DL111-B/2017) e...,None,None,1699278259
2,10324310,None,None,None,None,None,None,None,None,None,...,None,None,False,,True,None,Código dos Contratos Públicos ( DL 111-B/2017 ),None,None,1699279457
3,10323710,7795/2023,"2.400.000,00 €",30 dias.,False,6498116,13012738,15-05-2023,93,2,...,None,None,False,,True,None,Código dos Contratos Públicos (DL111-B/2017) e...,None,None,1699032550
4,10323720,None,None,None,None,None,None,None,None,None,...,None,None,False,,True,None,Código dos Contratos Públicos (DL111-B/2017) e...,None,None,1699032539
5,10323714,None,None,None,None,None,None,None,None,None,...,None,None,False,,True,None,Código dos Contratos Públicos (DL111-B/2017) e...,None,None,1699032543
6,10323516,11910/2023,"106.000,00 €",30 dias.,True,6751484,13549979,17-07-2023,137,2,...,None,None,False,,True,None,Código dos Contratos Públicos (DL111-B/2017) e...,None,None,1699032864
7,10323359,None,None,None,None,None,None,None,None,None,...,None,None,False,"Artigo 95.º, n.º 1, a), contrato de locação ou...",True,Compromisso N.º 4523702099,Código dos Contratos Públicos (DL111-B/2017) e...,None,None,1699033091
8,10323281,None,None,None,None,None,None,None,None,None,...,None,None,False,,True,None,Código dos Contratos Públicos (DL111-B/2017) e...,None,None,1699033192
9,10323708,None,None,None,None,None,None,None,None,None,...,None,None,False,"Artigo 95.º, n.º 1, c), locação ou aquisição d...",True,None,Código dos Contratos Públicos (DL111-B/2017) e...,None,None,1699032558
